In [86]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
from matplotlib import pyplot as plt
import io
from google.cloud import vision
from PIL import Image



In [87]:
path ='./image.jpg'

In [88]:
# 이미지를 로드합니다
img = cv2.imread(path)

In [89]:
# 분류를 위한 이미지 전처리를 수행합니다
image = cv2.resize(img, (280, 160))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

# 학습된 네트워크와 `MultiLabelBinarizer`를 로드합니다
print("[INFO] loading network...")
model = load_model("model.h5")
mlb = pickle.loads(open("labelbin", "rb").read())

# 이미지에 대한 분류를 수행한 후, 
# 확률이 가장 높은 두 개의 클래스 라벨을 찾습니다
# print("[INFO] classifying image...")
proba = model.predict(image)[0]
idxs = np.argsort(proba)[::-1][:2]

[INFO] loading network...


In [90]:
idxs

array([ 9, 13], dtype=int64)

In [91]:
for (i, j) in enumerate(idxs):
    label = "{}: {:.2f}%".format(mlb.classes_[j], proba[j] * 100)
    


In [92]:
first_acc=mlb.classes_[idxs[0]]
second_acc=mlb.classes_[idxs[1]]

print(first_acc)
print(second_acc)

circle
ellipse


In [100]:
import base64

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ='C:/test/crucial-lyceum-274205-eadd5d760574.json'

client = vision.ImageAnnotatorClient()

with io.open('./image.jpg', 'rb') as image_file:    
    content = image_file.read()
    
image = vision.types.Image(content=content)

response = client.text_detection(image=image)
texts = response.text_annotations

print(texts)
search_word=texts[2].description

[locale: "en"
description: "HD DWP\nmm\n9.3\n\354\213\235\355\222\210\354\235\230\354\225\275\355\222\210\354\225\210\354\240\204\354\262\230\nMinistry of Food and Drug Safety\n9.3\n"
bounding_poly {
  vertices {
    x: 23
    y: 116
  }
  vertices {
    x: 756
    y: 116
  }
  vertices {
    x: 756
    y: 410
  }
  vertices {
    x: 23
    y: 410
  }
}
, description: "HD"
bounding_poly {
  vertices {
    x: 88
    y: 116
  }
  vertices {
    x: 367
    y: 116
  }
  vertices {
    x: 367
    y: 235
  }
  vertices {
    x: 88
    y: 235
  }
}
, description: "DWP"
bounding_poly {
  vertices {
    x: 440
    y: 116
  }
  vertices {
    x: 675
    y: 116
  }
  vertices {
    x: 675
    y: 235
  }
  vertices {
    x: 440
    y: 235
  }
}
, description: "mm"
bounding_poly {
  vertices {
    x: 23
    y: 346
  }
  vertices {
    x: 55
    y: 346
  }
  vertices {
    x: 55
    y: 359
  }
  vertices {
    x: 23
    y: 359
  }
}
, description: "9.3"
bounding_poly {
  vertices {
    x: 202
    y:

In [101]:
search_word

'DWP'

In [102]:
import pandas as pd
df1=pd.read_excel('공공데이터개방_낱알식별목록_re.xlsx')

In [103]:
check1 = df1[df1['의약품제형'].str.contains(first_acc) | df1['색상앞'].str.contains(first_acc)]  
check2 = check1[check1['표시앞'].isin([search_word]) | check1['표시뒤'].isin([search_word])]
tablet_name =check2['품목명'].values
tablet_function=check2['분류명'].values

In [104]:
if not tablet_name:
    check1 = df1[df1['의약품제형'].str.contains(second_acc) | df1['색상앞'].str.contains(second_acc)]  
    check2 = check1[check1['표시앞'].isin([search_word]) | check1['표시뒤'].isin([search_word])]
    tablet_name =check2['품목명'].values
    tablet_function=check2['분류명'].values
    
   
    


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [84]:
print(tablet_name)
print(tablet_function)

[]
[]
